# Genetic Algorithm

Natural Selection based.

Initial Population -> Fitness Function -> Selection -> Crossover -> Mutation 

In [5]:
cities = [
    "Fortaleza",
    "Caucaia",
    "Juazeiro do Norte",
    "Maracanaú"
]
costs = [
    [0, 10, 100, 50],
    [10, 0, 20, 20],
    [100, 20, 0, 10],
    [50, 20, 10, 0]
]

# Idea: F -> C -> J -> M -> F


In [6]:
# Since we will always be back to initial state, doesn't seem smart to set it inside the DNA  
dna = "CJM"  # do this for every initial letter?

In [ ]:
gene_example = "CMJ"
best_gene = "CJM" 